In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Dropout,BatchNormalization,LSTM,Bidirectional,GlobalMaxPool1D,Input,Activation,Flatten,Embedding,Dense,concatenate,Conv1D,MaxPooling1D
import string
import re
from tqdm import tqdm
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score,classification_report
import os
from keras.models import Model
import seaborn as sns
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings('ignore')
import tensorboard
from textblob import TextBlob
import os
import tensorboard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime
from keras.initializers import he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable

In [2]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 35.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 49.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 27.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.2 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) 

In [3]:
data=pd.read_csv("../input/data-pre-6/train-balanced-sarcasm.csv")
data.shape

(1010826, 10)

In [4]:
null_values=data.isna().sum()
print(null_values)

label              0
comment           53
author             0
subreddit          0
score              0
ups                0
downs              0
date               0
created_utc        0
parent_comment     0
dtype: int64


In [5]:
data=data.dropna()
data.isna().sum()

label             0
comment           0
author            0
subreddit         0
score             0
ups               0
downs             0
date              0
created_utc       0
parent_comment    0
dtype: int64

In [6]:
duplicate_values=data[data.duplicated()]
print("duplicate rows in the dataset",len(duplicate_values))
data.drop_duplicates(keep='first',inplace=True)

duplicate rows in the dataset 28


In [7]:
data['score']=data['ups']-data['downs']

In [8]:
data_basic=data.drop(['author','score','ups','downs','date','created_utc','parent_comment','subreddit'],axis=1)

In [9]:
y=data_basic['label']
X=data_basic.drop(['label'],axis=1)

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,random_state=42,test_size=0.3,shuffle=True)
print((X_train.shape),y_train.shape)
print((X_test.shape),y_test.shape)

(707521, 1) (707521,)
(303224, 1) (303224,)


In [12]:
import ktrain
from ktrain import text
model_name = 'distilbert-base-uncased'
t = text.Transformer(model_name, maxlen=100, class_names=class_names)
trn = t.preprocess_train(list(X_train['comment']), y_train)
val = t.preprocess_test(list(X_test['comment']), y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=64)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 25
	99percentile : 38


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 25
	99percentile : 38


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [13]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
11056/11056 [==============================] - 4794s 419ms/step - loss: 0.5350 - accuracy: 0.7259 - val_loss: 0.5020 - val_accuracy: 0.7515
Epoch 2/5
11056/11056 [==============================] - 4772s 419ms/step - loss: 0.4819 - accuracy: 0.7656 - val_loss: 0.4837 - val_accuracy: 0.7646
Epoch 3/5
11056/11056 [==============================] - 4764s 418ms/step - loss: 0.4457 - accuracy: 0.7889 - val_loss: 0.4760 - val_accuracy: 0.7736
Epoch 4/5
11056/11056 [==============================] - 4774s 419ms/step - loss: 0.3823 - accuracy: 0.8273 - val_loss: 0.4925 - val_accuracy: 0.7707
Epoch 5/5
11056/11056 [==============================] - 4760s 418ms/step - loss: 0.3132 - accuracy: 0.8646 - val_loss: 0.5513 - val_accuracy: 0.7673


In [14]:
learner.model.save_pretrained('.// model_bert')

In [1]:
learner.validate(class_names=t.get_classes())

NameError: name 'learner' is not defined